### Columnas de Baños

In [390]:
import pandas as pd
import numpy as np
import re 

In [391]:
data_location = "./properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [392]:
data.shape

(121220, 26)

In [393]:
#Creación de una nueva columna 'baños' vacia, la cual va a contener la cantidad de baños en una propiedad
data['baños'] = np.nan

In [394]:
#Comprobación de valores null en 'baños'
data['baños'].isnull().value_counts()

True    121220
Name: baños, dtype: int64

In [395]:
#Cambio a minúscula estas dos nuevas columnas creadas
data['description_copia'] = data['description'].astype(str).str.lower()
data['title_copia']= data['title'].astype(str).str.lower()

In [396]:
#Comprobación de valores null en data 'description_copia'
data['description_copia'].isnull().value_counts()

False    121220
Name: description_copia, dtype: int64

In [397]:
#Comprobación de valores null en data 'title_copia'
data['title_copia'].isnull().value_counts()

False    121220
Name: title_copia, dtype: int64

#### Búsqueda de baños con expresiones regulares: 

In [398]:
patron = "\s(?P<numero>\d[\d]?)\s+baño[s]?"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'baños'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [399]:
data['baños'].isnull().value_counts()

True     111067
False     10153
Name: baños, dtype: int64

In [400]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
baños = pd.cut(data.baños, bins, right = False)
data.pivot_table('baños', index = [baños], aggfunc={'baños': 'count'})

,baños
baños,
"[0, 1)",0
"[1, 2)",3342
"[2, 3)",5068
"[3, 4)",1192
"[4, 5)",356
"[5, 6)",121
"[6, 7)",36
"[7, 8)",13
"[8, 9)",8


In [401]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [402]:
patron = "\s(?P<numero>\d[\d]?)\s+baã±o[s]?"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'baños'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [403]:
data['baños'].isnull().value_counts()

True     110519
False     10701
Name: baños, dtype: int64

In [404]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
baños = pd.cut(data.baños, bins, right = False)
data.pivot_table('baños', index = [baños], aggfunc={'baños': 'count'})

,baños
baños,
"[0, 1)",0
"[1, 2)",3623
"[2, 3)",5240
"[3, 4)",1246
"[4, 5)",375
"[5, 6)",133
"[6, 7)",42
"[7, 8)",14
"[8, 9)",8


In [405]:
patron = "\sbaño[s]?\s*:\s*(?P<numero>\d[\d]?)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'baños'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [406]:
data['baños'].isnull().value_counts()

True     110333
False     10887
Name: baños, dtype: int64

In [407]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
baños = pd.cut(data.baños, bins, right = False)
data.pivot_table('baños', index = [baños], aggfunc={'baños': 'count'})

,baños
baños,
"[0, 1)",0
"[1, 2)",3697
"[2, 3)",5320
"[3, 4)",1261
"[4, 5)",385
"[5, 6)",136
"[6, 7)",43
"[7, 8)",15
"[8, 9)",8


In [408]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [409]:
patron = r"\s(un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]*baño[s]?"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'un': 1,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [410]:
data.loc[mask_description_notnull,'baños']= encontradas1.astype(float)

In [411]:
data['baños'].isnull().value_counts()

True     100636
False     20584
Name: baños, dtype: int64

In [412]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
baños = pd.cut(data.baños, bins, right = False)
data.pivot_table('baños', index = [baños], aggfunc={'baños': 'count'})

,baños
baños,
"[0, 1)",0
"[1, 2)",10341
"[2, 3)",8046
"[3, 4)",1526
"[4, 5)",431
"[5, 6)",146
"[6, 7)",48
"[7, 8)",16
"[8, 9)",8


In [413]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [414]:
patron = r"\s(un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]*baã±o[s]?"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'un': 1,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [415]:
data.loc[mask_description_notnull,'baños']= encontradas1.astype(float)

In [416]:
data['baños'].isnull().value_counts()

True     99920
False    21300
Name: baños, dtype: int64

In [417]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
baños = pd.cut(data.baños, bins, right = False)
data.pivot_table('baños', index = [baños], aggfunc={'baños': 'count'})

,baños
baños,
"[0, 1)",0
"[1, 2)",10979
"[2, 3)",8122
"[3, 4)",1528
"[4, 5)",431
"[5, 6)",146
"[6, 7)",48
"[7, 8)",16
"[8, 9)",8


#### Ahora busco con las mismas regex pero con la columna 'title_copia'

In [418]:
patron = "\s(?P<numero>\d[\d]?)\s*baño[s]?"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'baños'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [419]:
data['baños'].isnull().value_counts()

True     99748
False    21472
Name: baños, dtype: int64

In [420]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
baños = pd.cut(data.baños, bins, right = False)
data.pivot_table('baños', index = [baños], aggfunc={'baños': 'count'})

,baños
baños,
"[0, 1)",0
"[1, 2)",10983
"[2, 3)",8246
"[3, 4)",1562
"[4, 5)",438
"[5, 6)",149
"[6, 7)",48
"[7, 8)",16
"[8, 9)",8


In [421]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"\sbaño[s]?"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.findall(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'un': 1,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [267]:
data.loc[data.description_copia != ""].sample(40)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,baños,description_copia,title_copia
113013,113013,sell,apartment,Nordelta,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4150913,-58.6394399",-34.415091,...,NaN,NaN,NaN,http://www.properati.com.ar/1c7sk_venta_depart...,Venta de Departamento 2 AMBIENTES en Q-Bay Isl...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/9/41y_WCjxzUaNSa...,NaN,venta de departamento 2 ambientes en q-bay isl...,departamento en venta
55024,55024,sell,house,Punta Indio,|Argentina|Buenos Aires Interior|Punta Indio|,Argentina,Buenos Aires Interior,3429692.0,"-35.3981091,-57.322414",-35.398109,...,NaN,3.0,NaN,http://www.properati.com.ar/19oa2_venta_casa_p...,CASA ZONA RINGUELET ( LA PLATA ),CASA RINGUELET,https://thumbs4.properati.com/1/c5MMAW_aYipSD_...,NaN,casa zona ringuelet ( la plata ),casa ringuelet
10479,10479,sell,apartment,Villa Crespo,|Argentina|Capital Federal|Villa Crespo|,Argentina,Capital Federal,3427458.0,"-34.595822,-58.438988",-34.595822,...,NaN,1.0,NaN,http://www.properati.com.ar/1685l_venta_depart...,Edificio ubicado en esquina cuya unidades dan ...,Departamento de 1 ambiente en Venta en Villa c...,https://thumbs4.properati.com/5/B5sJm86A05hOPi...,NaN,edificio ubicado en esquina cuya unidades dan ...,departamento de 1 ambiente en venta en villa c...
108137,108137,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|La ...,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,NaN,NaN,...,NaN,NaN,NaN,http://www.properati.com.ar/1bziq_venta_depart...,DEPARTAMENTO A ESTRENAR . MONOAMBIENTE CONTRAF...,Departamento - La Plata,https://thumbs4.properati.com/7/-elEHWDM7L_ty6...,NaN,departamento a estrenar . monoambiente contraf...,departamento - la plata
4845,4845,sell,house,Barracas,|Argentina|Capital Federal|Barracas|,Argentina,Capital Federal,3436134.0,"-34.6474429,-58.3788197",-34.647443,...,NaN,NaN,NaN,http://www.properati.com.ar/15ob5_venta_casa_b...,"Bellisima casa, totalmente reciclada a nuevo, ...",Casa en terreno propio,https://thumbs4.properati.com/2/GuL_ByQ7aL3sh1...,NaN,"bellisima casa, totalmente reciclada a nuevo, ...",casa en terreno propio
40367,40367,sell,apartment,Saavedra,|Argentina|Capital Federal|Saavedra|,Argentina,Capital Federal,3429422.0,"-34.5548727,-58.493826",-34.554873,...,NaN,3.0,NaN,http://www.properati.com.ar/18r6b_venta_depart...,Espectacular vista y ubicación,SAAVEDRA 3 AMB exc vista,https://thumbs4.properati.com/2/OOKIqX_hBi5y9b...,NaN,espectacular vista y ubicación,saavedra 3 amb exc vista
89563,89563,sell,apartment,Villa Ballester,|Argentina|Bs.As. G.B.A. Zona Norte|General Sa...,Argentina,Bs.As. G.B.A. Zona Norte,3427467.0,"-34.5443544,-58.5686386",-34.544354,...,NaN,NaN,NaN,http://www.properati.com.ar/1b8sx_venta_depart...,Espectacular emprendimiento de 47 unidades de...,Departamento - Chilavert,https://thumbs4.properati.com/1/OOwrGXF1Jz_OK_...,NaN,espectacular emprendimiento de 47 unidades de...,departamento - chilavert
64807,64807,sell,house,Rincón de Maschwitz,|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|Ri...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,...,NaN,NaN,NaN,http://www.properati.com.ar/1abaa_venta_casa_r...,Excelente casa!!!dentro Country Rincon de Masc...,Casa de excelente calidad de construcción,https://thumbs4.properati.com/6/x4Dp1NizP-ccRJ...,NaN,excelente casa!!!dentro country rincon de masc...,casa de excelente calidad de construcción
28207,28207,sell,house,Villa Devoto,|Argentina|Capital Federal|Villa Devoto|,Argentina,Capital Federal,3427451.0,"-34.6117282594,-58.5141399418",-34.611728,...,NaN,4.0,NaN,http://www.properati.com.ar/17vlf_venta_casa_v...,"Casa 4 ambiente, reciclada a nueva. Consta de:...","Casa 4 ambientes, garage, patio, terraza, quincho",https://thumbs4.properati.com/7/U8KgTfZMmnLRjl...,NaN,"casa 4 ambiente, reciclada a nueva.

In [271]:
data.loc[10439, 'description_copia']

'interesante casa en una planta, en muy buen estado de conservaciã³n, living y comedor, cocina con muebles de alto y bajo mesada, dos amplios dormitorios al contrafrente, baã±o con ducha. muy lindo jardã\xadn, gran galpã³n al fondo (posible cochera cubierta) con espacio para lavadero, entrada para varios autos.medidas de lote 10 x 23 m2 aprox'